# 네이버 view 검색을 활용한 블로그 검색

In [1]:
# 라이브러리 import
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #다음화면으로 빨리 넘어갈때
from selenium.common.exceptions import UnexpectedAlertPresentException #selenium 이용시 Alert을 제어
import urllib.request
import random
from selenium.webdriver.chrome.options import Options

In [4]:
# 크롤링 시작과 끝 날짜 (달 별로 크로링)

first_days = pd.date_range('2022/01/01','2022/12/06', freq='MS') # 각 달의 첫 날
last_days = pd.date_range('2022/01/01','2022/12/06', freq='M') # 각 달의 마지막날

keyword_list = ['제니인스타그램'] #검색 키워드 추가


In [3]:
keyword_main = []
keyword_sub = []


keyword_list = []
for i in keyword_main:
    for w in keyword_sub:
        keyword_list.append(i + " " + w)
keyword_list = keyword_main + keyword_sub
keyword_list

[]

In [5]:
# 크롬드라이버 제어
chrome_options = Options()
chrome_options.add_argument('--no-sandbox') #bypass OS security model
chrome_options.add_argument('--disable-dex-shm-usage')#overcome limited resource problems
# options.add_argument('--headless') # 브라우저 백그라운드 모드


chrome_path = 'chromedriver.exe' #현재 컴퓨터의 크롬드라이버 위치

In [6]:
# 블로그 주소 수집

blog_url_list = []

#네이버에 키워드 검색
for keyword in keyword_list:
    url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=blog&query={}&oquery={}'.format(keyword, keyword)
        
    driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
    driver.implicitly_wait(3)
    
    driver.get(url)
    time.sleep(3)
    
    url_want = 990 #
    
    driver.find_element('xpath','//*[@id="snb"]/div[1]/div/div[2]/a').click() #검색 옵션 클릭

    #날짜 지정
    for k in range(len(first_days)):
        try:
            #시작과 끝의 년월일 설정
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[9]/i').click() #직접 입력
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(first_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(first_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(first_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[1]/span[3]/a').click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[1]/div/div/div/ul/li[{0}]'.format(last_days.year[k]-2002)).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[2]/div/div/div/ul/li[{0}]/a'.format(last_days.month[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[2]/div[3]/div/div/div/ul/li[{0}]/a'.format(last_days.day[k])).click()
            driver.find_element('xpath','//*[@id="snb"]/div[2]/ul/li[3]/div/div[2]/div[3]/button').click()            
            
            
            # 스크롤 조작
            last_height = driver.execute_script('return document.body.scrollHeight')
            
            scroll = (url_want/30)-1 #한 페이지에 기본으로 30개 글이 나옴
            
            # 스크롤 다운
            for i in range(int(scroll)):
                driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
                time.sleep(random.uniform(1,1.7))
                
                new_height = driver.execute_script('return document.body.scrollHeight')
                
                if new_height == last_height:
                    break
                    
                last_height = new_height
                
                
            soup = bs(driver.page_source, 'lxml')
            blog_url = soup.find_all('a',class_='api_txt_lines total_tit')
            
            # url 주소 수집
            for i in blog_url:
                blog_url_list.append(i['href']) #주소 수집
                
            print('{0} 키워드 {1}년{2}월'.format(keyword, 
                                            first_days.year[k], first_days.month[k]), 
                  len(blog_url),'개 블로그 url 주소 수집')
            
            
            driver.execute_script('window.scrollTo(0,0)')
            
        except:
            pass
            

C:\Users\USER2\AppData\Local\Temp\ipykernel_12948\311011288.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)
C:\Users\USER2\AppData\Local\Temp\ipykernel_12948\311011288.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options)


직장인일기 키워드 2022년1월 990 개 블로그 url 주소 수집
직장인일기 키워드 2022년2월 990 개 블로그 url 주소 수집
직장인일기 키워드 2022년3월 990 개 블로그 url 주소 수집
직장인일기 키워드 2022년4월 150 개 블로그 url 주소 수집
직장인일기 키워드 2022년5월 990 개 블로그 url 주소 수집
직장인일기 키워드 2022년6월 990 개 블로그 url 주소 수집
직장인일기 키워드 2022년7월 990 개 블로그 url 주소 수집
직장인일기 키워드 2022년8월 120 개 블로그 url 주소 수집
직장인일기 키워드 2022년9월 990 개 블로그 url 주소 수집
직장인일기 키워드 2022년10월 990 개 블로그 url 주소 수집


In [7]:
blog_url_list #잘 저장 되었는지 확인

['https://blog.naver.com/hyelim_26/222615680705',
 'https://blog.naver.com/haruming_r/222625749550',
 'https://blog.naver.com/hyejinjjong/222627160087',
 'https://blog.naver.com/helly644/222621075953',
 'https://blog.naver.com/grace03160/222622955020',
 'https://blog.naver.com/eselium07/222610523768',
 'https://blog.naver.com/wjstmddus95/222612557159',
 'https://blog.naver.com/91_hanna/222611954791',
 'https://blog.naver.com/tember909/222614673814',
 'https://blog.naver.com/ls5858/222618148884',
 'https://blog.naver.com/gonirao/222617696101',
 'https://zepero.com/2227',
 'https://blog.naver.com/dearhye_/222628156944',
 'https://blog.naver.com/hyelim_26/222633302960',
 'https://blog.naver.com/agua-nov/222635022983',
 'https://blog.naver.com/ahbsm/222633760811',
 'https://blog.naver.com/helly644/222629495066',
 'https://blog.naver.com/grace03160/222616555633',
 'https://blog.naver.com/eselium07/222623153038',
 'https://blog.naver.com/wjstmddus95/222625260472',
 'https://blog.naver.com/91

In [8]:
len(blog_url_list) #수집된 url 갯수 확인

8190

In [9]:
# 수집 후 url list 중복 확인

real_list = set(blog_url_list)
blog_url_list = list(real_list)
len(blog_url_list)

8190

In [10]:
# 파일 저장(블로그 주소)

k = pd.DataFrame(blog_url_list)
k.to_csv('url_list_스마트폰앨범정리.csv', encoding='utf-8-sig') #파일 이름만 바꿔주시면 됩니다. "url_list_myname_alone"

In [55]:
# 블로그 크롤링
url = pd.read_csv('url_list_스마트폰앨범정리.csv')
blog_url_list = list(url['0'])

In [56]:
no_naver_blog1=[]

start = time.time()


time_list = [] #시간
review_list = [] #리뷰
comment_list = [] #댓글
like_list = [] #좋아요수
url_list = [] #url주소
count = 0

driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
for url in blog_url_list: #기본적으로 고치지 않으시지만, n번째 크롤링에서 멈췄다면, blog_url_list를 blog_url_list[n:]으로 고칩니다.
    if 'naver' in url: #네이버 블로그만 수집(다른 사이트가 섞이면 크롤링 에러가 남)
    
        count += 1
        driver.get(url) #url 하나씩 글을 수집할꺼야
        time.sleep(1)

        try:
            # 전체 본문 가지고 와서
            driver.switch_to.frame('mainFrame') #네이버는 mainFrame을 바꿔줘야 수집이 됌
            time.sleep(1)
        
            soup = bs(driver.page_source, 'lxml') #뷰티풀숩을 이용해서 구문 분석(lxml 파싱 이용. 모든 테그에서 데이터를 추출위함)
            postview = soup.find('div', attrs={'id': re.compile('post-view.+')}).get_text()

            reg = re.compile(r'[\s+]') #r은 raw string. \s+는 공백문자
            postview_reg = reg.sub(' ',postview) #공백을 띄어쓰기로 대치해서 본문 글 수집
            try:
                # 좋아요수
                like = driver.find_element("xpath", "//span[@class='u_likeit_list_btn _button btn_sympathy pcol2 off']/em[@class='u_cnt _count']")
                like = like.text
            except:
                like = [] #좋아요 없으면 빈 리스트
            
            try:
                timeline = driver.find_element("xpath", "//span[@class='se_publishDate pcol2']") #수정 안 한 경우
        
            except:
                timeline = driver.find_element("xpath", "//p[@class='date fil5 pcol2 _postAddDate']") #수정 한 경우
            timeline = timeline.text

            try:
                driver.find_element(By.CSS_SELECTOR,'span.btn_arr').click() #댓글 내리는 버튼(태그명.클래스선택자)
                #driver.find_element_by_css_selector('span.btn_arr').click()
                time.sleep(1.7)

                comment_blog = []
                comment = driver.find_element(By.CSS_SELECTOR,'span.u_cbox_contents') #댓글
                #comment = driver.find_elements_by_css_selector('span.u_cbox_contents')
                for i in comment: #댓글 수집
                    com = str(i.text) #
                    com = reg.sub(' ',com) 
                    comment_blog.append(([com]))
            except: 
                comment_blog = []
             
            time_list.append(timeline)
            review_list.append(postview_reg)
            comment_list.append(comment_blog)
            url_list.append(url)
            like_list.append(like)
            time.sleep(random.uniform(1,1.6))
        except UnexpectedAlertPresentException:
            pass
    else:
        no_naver_blog1.append(url)
        print('네이버 블로그 아닌 url: ',url)
        
    if count%10 ==0:
        print('{}개 블로그 크롤링 완료'.format(count))

C:\Users\USER2\AppData\Local\Temp\ipykernel_14696\3172629000.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.
C:\Users\USER2\AppData\Local\Temp\ipykernel_14696\3172629000.py:13: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=chrome_path, chrome_options=chrome_options) #크롬 드라이버 위치만 바꿔주시면 됩니다.


네이버 블로그 아닌 url:  https://hyunjee-ization.tistory.com/19
10개 블로그 크롤링 완료
20개 블로그 크롤링 완료


KeyboardInterrupt: 

In [57]:
len(time_list)

28

In [59]:
data = {'time':time_list,
       'review':review_list,
       'comment':comment_list,
       'like':like_list,
       'url':url_list}

df = pd.DataFrame(data)

print('크롤링 소요 시간: ',time.time()-start)
print('총 {}개 블로그 크롤링 완료'.format(count))
df.head()

크롤링 소요 시간:  399.93249583244324
총 28개 블로그 크롤링 완료


,time,review,comment,like,url
0,2021. 1. 2. 15:28,안녕? 임신일기 7 유샤루미 ・ 2021. 1...,[],[],https://blog.naver.com/skysky006/222193684992
1,2021. 1. 16. 23:23,교정일기 [치아교정일기3] 치과 상담 및 정밀검사 후기 ...,[],4,https://blog.naver.com/ne0822/222209734340
2,2021. 1. 30. 17:27,책 이야기 무엇을 해야할지 모르겠을 때 떠난 [세계로 나간 일...,[],2,https://blog.naver.com/bandi07289/222225431386
3,2021. 1. 20. 9:20,• 밥상일기 [밥상일기] 신혼부부 2인 식단 / 2021년 1...,[],19,https://blog.naver.com/inthemoodforlove__/2222...
4,2021. 1. 12. 13:32,"그냥관심사 [ 주식일기 ] 주식 ""시간외거래"" 하는법 (시간외...",[],29,https://blog.naver.com/janmang91/222204876957


In [60]:
#csv 저장
df.to_csv("제니인스타그램.csv", encoding = "utf-8-sig") 